In [1]:
import gensim
import pandas as pd
import numpy as np
import sklearn
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

def normalize(vector):
    normalized_vector = vector / np.linalg.norm(vector)
    return normalized_vector

def dimension(model, positives, negatives):
    diff = sum([normalize(model[x]) for x in positives]) - sum([normalize(model[y]) for y in negatives])
    return diff

def makeDF(model, dim, word_list):
    good_to_bad = []
    for word in word_list:
        good_to_bad.append(sklearn.metrics.pairwise.cosine_similarity(model[word].reshape(1,-1), dim.reshape(1,-1))[0][0])
    df = pd.DataFrame({'good_to_bad': good_to_bad}, index = word_list)
    return df

def project_word(model, pos_lst, neg_lst, projecting_words):
    dim = dimension(model, pos_lst, neg_lst)
    result_df = makeDF(model, dim, projecting_words)
    result_df = result_df.sort_values(by = ['good_to_bad'], ascending= False)
    return (result_df)

pos_lst = ['good', 'better', 'right', 'satisfactory', 'positive', 'effective', 'sufficient', 'excellent', 'success']
neg_lst = ['bad', 'worse', 'wrong', 'unsatisfactory', 'negative', 'ineffective', 'insufficient', 'failed', 'failure']

quant_words = ["measure", 'result', 'performance', 'hypothesis', 'finding']
qual_words = ["political", "social", "historical", 'theory', 'context']
total_words = quant_words + qual_words

In [2]:
qual_model = gensim.models.Word2Vec.load("qual_model")
qual_proj = project_word(qual_model, pos_lst, neg_lst, total_words)

In [3]:
quant_model = gensim.models.Word2Vec.load("quant_model")
quant_proj = project_word(quant_model, pos_lst, neg_lst, total_words)

In [4]:
def compute_corr(proj_1, proj_2):
    proj_1['order'] = [i for i in range(1, len(proj_1) + 1)]
    proj_2['order'] = [i for i in range(1, len(proj_2) + 1)]
    
    proj_1_alphabet_order = proj_1.sort_index()
    proj_2_alphabet_order = proj_2.sort_index()

    pearson_corr = proj_1_alphabet_order['good_to_bad'].corr(proj_2_alphabet_order['good_to_bad'])
    rank_corr = proj_1_alphabet_order['order'].corr(proj_2_alphabet_order['order'], method = 'spearman')
    return (pearson_corr, rank_corr)

compute_corr(qual_proj, quant_proj)

(-0.5969539798258481, -0.6969696969696969)

In [5]:
fig_2_word_lst = []
with open("Fig_2_words.tsv", 'r') as f:
    for line in f:
        fig_2_word_lst.append(line.rstrip("\n"))

In [6]:
quant_fig_2 = project_word(quant_model, pos_lst, neg_lst, fig_2_word_lst)
qual_fig_2 = project_word(qual_model, pos_lst, neg_lst, fig_2_word_lst)

compute_corr(quant_fig_2, qual_fig_2)

(-0.36806088195529324, -0.33795034079844205)